# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize, FreqDist

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
#Удаляю лишний столбец
data = data.drop(['Unnamed: 0'], axis=1)
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [5]:
print('Количество нетоксичных и токсичных комментариев:')
print(data['toxic'].value_counts())
print(70 * '-')
print('Количество пропусков:')
print(data.isna().sum())
print(70 * '-')
print('Количество дубликатов:', data.duplicated().sum())

Количество нетоксичных и токсичных комментариев:
0    143106
1     16186
Name: toxic, dtype: int64
----------------------------------------------------------------------
Количество пропусков:
text     0
toxic    0
dtype: int64
----------------------------------------------------------------------
Количество дубликатов: 0


In [6]:
#Отношение нетоксичных комментариев к токсичным
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_ratio

8.841344371679229

In [7]:
#Функция для замены регулярных выражений и понижения регистра
def lemmatize_text(text):
    text = text.lower()
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', text) 
    return " ".join(cleared_text.split())

data['lower_clear_text'] = data['text'].apply(lemmatize_text)

data = data.drop(['text'], axis=1)

In [8]:
data

,toxic,lower_clear_text
0,0,explanation why the edits made under my userna...
1,0,d aww he matches this background colour i m se...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestions on impr...
4,0,you sir are my hero any chance you remember wh...
...,...,...
159287,0,and for the second time of asking when your vi...
159288,0,you should be ashamed of yourself that is a ho...
159289,0,spitzer umm theres no actual article for prost...
159290,0,and it looks like it was actually you who put ...


In [9]:
L = WordNetLemmatizer()

In [10]:
#Функция выполняет токенизациию и лемматизацию массива текстов
def lemmatizered(corpus):
    corpus_new = []
    for sentence in corpus:
        word_list = nltk.word_tokenize(sentence)
        corpus_new.append(' '.join([L.lemmatize(w) for w in word_list]))
    return corpus_new

In [11]:
nltk.download('stopwords')
stopwords.words('english')[:5]

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we']

In [12]:
#Функция возвращает словарь, где возвращается значение часть речи (pos_tag)
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
 
    return tag_dict.get(tag, wordnet.NOUN)

In [13]:
#Функция выполняет токенизациию и лемматизацию массива текстов c учетом pos_tag и удаление стоп-слов
def get_word_text(corpus):
    corpus_new = []
    for sentence in corpus:
        corpus_new.append(' '.join([L.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence) if not w in stopwords.words('english')]))
    return corpus_new

In [14]:
%%time
data['lemma_text_no_sw'] = get_word_text(data['lower_clear_text'])

CPU times: user 1.77 s, sys: 145 ms, total: 1.92 s
Wall time: 2.05 s


In [15]:
data = data.drop(['lower_clear_text'], axis=1)
data.head()

,toxic,lemma_text_no_sw
0,0,explanation edits make username hardcore metal...
1,0,aww match background colour seemingly stuck th...
2,0,hey man really try edit war guy constantly rem...
3,0,make real suggestion improvement wonder sectio...
4,0,sir hero chance remember page


In [ ]:
#После преобразований появились дубликаты, удаляю их
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.duplicated().sum()

In [ ]:
#Деление на выборки
features = data.drop(['toxic'], axis=1)
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.2, 
                                                                              random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, 
                                                                            target_train, 
                                                                            test_size=0.25,
                                                                            random_state=12345)

In [ ]:
#Подключаю стоп-слова и преобразую признаки в тип Unicode
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values)
features_valid = count_tf_idf.transform(features_valid['lemm_text'].values)
features_test = count_tf_idf.transform(features_test['lemm_text'].values)

cv_counts = 3

In [ ]:
%%time
#Проверка модели на обучающих выборке с помощью кросс-валидации
classificator = LogisticRegression()
train_f1 = cross_val_score(classificator, 
                      features_train, 
                      target_train, 
                      cv=cv_counts, 
                      scoring='f1').mean()
print('F1 на cv:', train_f1)

In [ ]:
%%time
#Так как данные не сбалансированы применяю балансировку
classificator = LogisticRegression(class_weight='balanced')
train_f1_ballanced = cross_val_score(classificator, 
                                    features_train, 
                                    target_train, 
                                    cv=cv_counts, 
                                    scoring='f1').mean()
print('F1 на cv с балансированными классами:', train_f1_ballanced)

In [ ]:
%%time

dict_classes={0:1, 1:class_ratio}
classificator = LogisticRegression(class_weight=dict_classes)
train_f1_ballanced = cross_val_score(classificator, 
                                    features_train, 
                                    target_train, 
                                    cv=cv_counts, 
                                    scoring='f1').mean()
print('F1 на cv с балансированными классами:', train_f1_ballanced)

#F1 поднялась, но в первом случае недостаточно, так что дальше использую второй метод

<b>Вывод</b>  
После просмотра информации о данных оказалось, что есть не нужный столбец, который был удален. В целевом признаке присутствует дисбаланс - токсичных комментариев почти в 10 раз меньше, чем не токсичных. До обработки текста дубликатов не было, после замены регулярных выражений и лемматизации появились дубликаты, которые были удалены. Также данные были разбиты на выборки (обучающие, валидационные, тестовые). Из них были убраны стоп-слова и тип приведен к Unicode. Проверил F1 на обучающих данных. Результат не соответствует поставленной задаче, поэтому классы в целевом признаке были сбалансированы, что привело к повышению F1.

## Обучение

In [ ]:
%%time
#Подбираю лучшие параметры для логистической регрессии
classificator = LogisticRegression()
hyperparams = [{'solver':['lbfgs', 'liblinear'],
                'C':[0.1, 1, 10],
                'class_weight':[dict_classes]}]

clf = GridSearchCV(classificator, hyperparams, scoring='f1',cv=cv_counts)
clf.fit(features_train, target_train)
print('Лучшие параметры:')
LR_best_params = clf.best_params_
print(LR_best_params)
print(80*'-')
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
print(80*'-')
cv_f1_LR = max(means)
print('F1 на cv:', cv_f1_LR)
print(80*'-')

In [ ]:
%%time
#F1 на валидации
classificator = LogisticRegression()
classificator.set_params(**LR_best_params)
classificator.fit(features_train, target_train)
target_predict = classificator.predict(features_valid)
valid_f1_LR = f1_score(target_valid, target_predict)
print('F1 на валидации:', valid_f1_LR)
print(80*'-')

In [ ]:
%%time
#Подбор параметров для дерева решений
classificator = DecisionTreeClassifier()
hyperparams = [{'max_depth':[x for x in range(30,51,2)],
                'random_state':[12345],
                'class_weight':[dict_classes]}]

clf = GridSearchCV(classificator, hyperparams, scoring='f1',cv=cv_counts)
clf.fit(features_train, target_train)
print('Лучшие параметры:')
DTC_best_params = clf.best_params_
print(DTC_best_params)
print(80*'-')
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
print(80*'-')    
cv_f1_DTC = max(means)
print('F1 на cv:', cv_f1_DTC)
print(80*'-')

In [ ]:
%%time
#F1 на валидации
classificator = DecisionTreeClassifier()
classificator.set_params(**DTC_best_params)
classificator.fit(features_train, target_train)
target_predict = classificator.predict(features_valid)
valid_f1_DTC = f1_score(target_valid, target_predict)
print('F1 на валидации', valid_f1_DTC)

In [ ]:
%%time
#Применение CatBoostClassifier
classificator = CatBoostClassifier(verbose=False, iterations=200)
classificator.fit(features_train, target_train)
target_predict = classificator.predict(features_valid)
cv_f1_CBC = cross_val_score(classificator,
                                         features_train, 
                                         target_train, 
                                         cv=cv_counts, 
                                         scoring='f1').mean()
valid_f1_CBC = f1_score(target_valid, target_predict)
print('F1 на cv:', cv_f1_CBC)
print('F1 на валидации:', valid_f1_CBC)

<b>Вывод</b>  
Были рассмотрены 3 модели: LogisticRegression, DecisionTreeClassifier, CatBoostClassifier. По F1 лучше всех себя показала LogisticRegression - результат 0.7698 на валидационной выборке, далее CatBoostClassifier с результатом 0.7522 на валидационной выборке. Хуже всех DecisionTreeClassifier - 0.6358 на валидационной выборке. На тестовой выборке буду использовать LogisticRegression.   

## Выводы

In [ ]:
#Проверка лучшей модели на тестовой выборке
classificator = LogisticRegression()
classificator.set_params(**LR_best_params)
classificator.fit(features_train, target_train)
predict_test = classificator.predict(features_test)
print('F1:', f1_score(target_test, predict_test))

<b>Вывод</b>  
На тестовой выборке LogisticRegression показала результат 0.7599, что соответствует условию задачи.